In [12]:
from __future__ import print_function, division
import pprint
import os 
import fnmatch
#import errno
import os.path as osp
import json
#import re
import glob as gb
from six import string_types
from nipype.interfaces.base import Bunch

import utils
reload(utils)

<module 'utils' from 'utils.py'>

In [13]:
pp = pprint.PrettyPrinter(indent=2)

In [14]:
#- testing

test_case_105a = {
    'base_dir': '/home/jb/data/bids/ds105',
    'model_pattern': "ds-105_sub*_model.json"
}
test_case_105b = {
    'base_dir': '/home/jb/data/bids/ds105',
    'model_pattern': "ds-105_level-all_model.json"
}
test_case_005 = {
    'base_dir': '/home/jb/data/bids/ds005',
    'model_pattern': "*_model.json"
}

test_case = test_case_005
test_case = test_case_105a
#test_case = test_case_105b

base_dir, model_pattern = test_case['base_dir'], test_case['model_pattern'] 

all_jsons = utils.glob_recursive(base_dir, model_pattern)
print(all_jsons)
json_model=all_jsons[0]

['/home/jb/data/bids/ds105/sub-2/func/models/ds-105_sub-2_model.json', '/home/jb/data/bids/ds105/models/ds-105_sub-all_model.json']


In [15]:
a, b, c = utils._get_nipype_specify_model_inputs(base_dir, model_pattern)

['/home/jb/data/bids/ds105/sub-6/func/sub-6_task-objectviewing_run-06_bold.nii.gz',
 '/home/jb/data/bids/ds105/sub-1/func/sub-1_task-objectviewing_run-01_bold.nii.gz',
 '/home/jb/data/bids/ds105/sub-3/func/sub-3_task-objectviewing_run-03_bold.nii.gz',
 '/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-03_bold.nii.gz',
 '/home/jb/data/bids/ds105/sub-1/func/sub-1_task-objectviewing_run-06_bold.nii.gz',
 '/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-02_bold.nii.gz',
 '/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-08_bold.nii.gz',
 '/home/jb/data/bids/ds105/sub-6/func/sub-6_task-objectviewing_run-01_bold.nii.gz',
 '/home/jb/data/bids/ds105/sub-4/func/sub-4_task-objectviewing_run-10_bold.nii.gz',
 '/home/jb/data/bids/ds105/sub-3/func/sub-3_task-objectviewing_run-04_bold.nii.gz',
 '/home/jb/data/bids/ds105/sub-6/func/sub-6_task-objectviewing_run-03_bold.nii.gz',
 '/home/jb/data/bids/ds105/sub-4/func/sub-4_task-objectviewing_run-01_bold.n

In [142]:
data = utils.get_json_model_Ydata(all_jsons[0], level='Run', verbose=utils.VERB['info'])

base dir /home/jb/data/bids/ds105/sub-2/func
json_model /home/jb/data/bids/ds105/sub-2/func/models/ds-105_sub-2_model.json


In [143]:
pp.pprint(data[:min(10, len(data))])

[ '/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-03_bold.nii.gz',
  '/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-05_bold.nii.gz',
  '/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-12_bold.nii.gz',
  '/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-02_bold.nii.gz',
  '/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-10_bold.nii.gz',
  '/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-08_bold.nii.gz',
  '/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-07_bold.nii.gz',
  '/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-04_bold.nii.gz',
  '/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-11_bold.nii.gz',
  '/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-09_bold.nii.gz']


In [144]:
model_pattern = "*_model.json"
model_data = utils.get_funcs_models(base_dir, model_pattern, verbose=utils.VERB['none'])

In [145]:
datafile = data[0]
model_dict = model_data[datafile]
pp.pprint(model_dict)

{ u'Columns': { u'face': { u'HRFmodelling': u'Gamma',
                           u'Level': u'face',
                           u'Variable': u'trial_type'},
                u'house': { u'HRFmodelling': u'Gamma',
                            u'Level': u'house',
                            u'Variable': u'trial_type'}},
  u'Contrasts': { u'face_vs_house': { u'Columns': [u'face', u'house'],
                                      u'Weights': [1, -1]},
                  u'house_vs_face': { u'Columns': [u'face', u'house'],
                                      u'Weights': [1, -1]}},
  u'DependentVariable': u'task-objectviewing',
  u'Level': u'Run',
  u'Name': u'run_level_simple'}


In [146]:
import pandas as pd
tsv_file = datafile.split('_bold.nii')[0] + '_events.tsv'
assert osp.isfile(tsv_file), "{} is not a file ".format(tsv_file)
print(tsv_file)

/home/jb/data/bids/ds105/sub-2/func/sub-2_task-objectviewing_run-03_events.tsv


In [147]:
dic = utils._get_json_dict_from_tsv_file(tsv_file)
#utils._get_tsv_lines(dic, 'trial_type', 'chair')

In [148]:
model_dict = model_data[datafile]
print(model_dict.keys())
regressors = model_dict['Columns']
pp.pprint(regressors)

[u'Contrasts', u'DependentVariable', u'Name', u'Columns', u'Level']
{ u'face': { u'HRFmodelling': u'Gamma',
             u'Level': u'face',
             u'Variable': u'trial_type'},
  u'house': { u'HRFmodelling': u'Gamma',
              u'Level': u'house',
              u'Variable': u'trial_type'}}


In [149]:
list_dicts = utils.get_run_conditions(datafile, model_dict)
condition_names = list_dicts.keys()

In [150]:
#pp.pprint(list_dicts)
for datafile in data[:min(10, len(data))]:
    #print(datafile)
    try: 
        nipype_info = utils.get_nipype_run_info(datafile, model_dict, verbose=utils.VERB['none'])
    except:
        print(datafile, ' : has a pb ... a condition has no event ')

In [151]:
#print(datafile)
#pp.pprint(nipype_info)
#print(nipype_info)

In [152]:
dict_regressors = utils.get_run_conditions(datafile, model_dict)#, verbose=verbose)
condition_names = dict_regressors.keys() 
#print(dict_regressors)
bunch = utils.make_nipype_bunch(dict_regressors)

In [153]:
bunch

Bunch(conditions=[u'house', u'face'], durations=[[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]], onsets=[[120.0, 122.0, 124.0, 126.0, 128.0, 130.0, 132.0, 134.0, 136.0, 138.0, 140.0, 142.0], [12.0, 14.0, 16.0, 18.0, 20.0, 22.0, 24.0, 26.0, 28.0, 30.0, 32.0, 34.0]], pmod=[[], []])

### copy a ds tree with only the json and tsv files

In [154]:
source_dir = '/home/jb/data/bids/ds005'
dest_dir = '/home/jb/code/bids2pype/bids2pype/tests/data/ds005'
# utils.create_empty_bids(source_dir, dest_dir, ['*.json','*.tsv'])

In [155]:
utils._sorted_by_depth('/home/jb/data/bids/ds105', "*sub*_model.json")

['/home/jb/data/bids/ds105/models/ds-105_sub-all_model.json',
 '/home/jb/data/bids/ds105/sub-2/func/models/ds-105_sub-2_model.json']

In [157]:
res = utils._get_substr_between(datafile, 'task-', '_')
print(res)

objectviewing
